# Introduction

The purpose of this code is to take the Orlando and Seattle weather data and clean and join them into one tidy data frame that only has the relavant precipation data. Precipitation data is in inches and consists of 5 years (2018-2022) and comes from one weather station in each city. Original data comes from NOAA https://www.ncei.noaa.gov/cdo-web/search;jsessionid=F63B263F76CBED355E2DFC8D9511FDB2 

# Data Sources and Import

In [3]:
# Import pandas, numpy, and matplotlib
import pandas as pd
import numpy as np
     

In [4]:
# load data sets for Seattle and Orlando precipitation
df_seattle = pd.read_csv(
    'https://raw.githubusercontent.com/naomi-rlm/Weather-Project/main/data/seattle_rain.csv'
)
df_orlando = pd.read_csv(
     'https://raw.githubusercontent.com/naomi-rlm/Weather-Project/main/data/orlando_rain.csv'
)

# Convert Data Types

In [5]:
# fix date data type
df_seattle['DATE'] = pd.to_datetime(df_seattle['DATE']) 
df_orlando['DATE'] = pd.to_datetime(df_orlando['DATE']) 

C:\Users\naomi\AppData\Local\Temp\ipykernel_17944\359811099.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_seattle['DATE'] = pd.to_datetime(df_seattle['DATE'])


# Joining Dataframes

In [6]:
# outer merge with the key DATE keeping the columns DATE and PRCP
df = df_orlando[['DATE', 'PRCP']].merge(df_seattle[['DATE', 'PRCP']], on='DATE', how='outer')

In [7]:
# transform data frame to have data under columns 'city' and 'precipitation' while still keeping 'DATE'
df = pd.melt(df, id_vars='DATE', var_name='city', value_name='precipitation')

In [8]:
# checking to make sure join was succesful
df.head()

,DATE,city,precipitation
0,2018-01-01,PRCP_x,0.04
1,2018-01-02,PRCP_x,0.01
2,2018-01-03,PRCP_x,1.02
3,2018-01-04,PRCP_x,0.00
4,2018-01-05,PRCP_x,0.00


# Renaming for Tidy Data

In [9]:
# change PRCP_x and PRCP_y to their actual city codes
df.loc[df['city'] == 'PRCP_x', 'city'] = 'ORL'
df.loc[df['city'] == 'PRCP_y', 'city'] = 'SEA'

In [10]:
# decapitaize 'DATE' column to make data frame column names more uniform
df = df.rename(columns={'DATE': 'date'})

# Handle Missing Values in Dataframe

In [11]:
# determining the number NAs in the data frame for Seattle and Orlando
df.isna().sum()

date               0
city               0
precipitation    192
dtype: int64

In [12]:
# NAs just for Seattle
df.loc[df['city'] == 'SEA', 'precipitation'].isna().sum()

np.int64(190)

In [13]:
#NAs just for Orlando
df.loc[df['city'] == 'ORL', 'precipitation'].isna().sum()

np.int64(2)

In [14]:
# define a column that labels each day by the day of the year
df['day_of_year'] = pd.DatetimeIndex(df['date']).day_of_year

In [18]:
# mean precipitation for each day in Seattle, averaged across years
# replace missing days for each city with their average
mean_day_by_city = (df.groupby(['city','day_of_year'])['precipitation'].mean())
df.loc[df['precipitation'].isna(),'precipitation'] = (
    df[df['precipitation'].isna()].apply(lambda r: mean_day_by_city.loc[(r['city'], r['day_of_year'])], axis=1)
)

In [19]:
# making sure NAs have all been replaced
df.isna().sum()

date             0
city             0
precipitation    0
day_of_year      0
dtype: int64

In [22]:
# exporting tidy df
df.to_csv('clean_seattle_orlando_weather.csv', encoding='utf-8-sig', index=False)

In [23]:
import os
os.getcwd()

'C:\\Users\\naomi\\Desktop\\DATA5100\\weather\\code'